# Método backward

En la implementación de la capa `Bias` las fórmulas de las derivadas eran relativamente sencillas, y la complejidad estaba más que todo en cómo utilizar el framework y comprender la diferencia entre la derivada de la entrada y la de los parámetros.

El método backward de la capa `Linear` requiere calcular $\frac{dE}{dy}$ y $\frac{dE}{dw}$. En términos de uso del framework, la implementación es muy similar a la de `Bias`, pero las fórmulas de las derivadas son más complicadas.

Primero asumiremos que hay un solo ejemplo de entrada $x$, para simplificar el desarrollo, y luego generalizaremos a un lote de $N$ ejemplos.


## $dE / dx$

Comenzamos con el caso de $\frac{dE}{dx}$. Si bien este caso es en realidad simétrico con respecto a $\frac{dE}{dw}$, es un poco más fácil de atacar conceptualmente.

Vamos a pensar en esta derivada por casos, desde el más simple al más complejo, incrementando las dimensiones de entrada y salida.


### 1 entrada, 1 salida

Comenzamos por en el caso más simple, donde tanto la entrada como la salida son 1D, entonces $x \in R$ y $w \in R$, es decir, son escalares. Entonces $\frac{dE}{dy}$ también es un escalar, y por regla de la cadena:

$\frac{dE}{dx} = \frac{dE}{dy} \frac{dy}{dx} = \frac{dE}{dy} \frac{d(wx)}{dx} = \frac{dE}{dy} w$



### I entradas, 1 salida

Pasemos ahora al caso con $I$ entradas y 1 salida. Entonces $x$ es un vector de $I$ valores, es decir, $x \in R^I$ y por ende $w \in R^I$ también es un vector con $I$ valores. En tal caso, podemos pensar la salida como el producto punto o matricial entre $w$ y $x$

$y = x . w = \sum_{i=1}^I x_i w_i$

Entonces, tenemos una derivada parcial por cada entrada: $\frac{dE}{dx_j}$. Recordando que $\frac{dE}{dy}$ sigue siendo un escalar (porque hay una sola salida), y utilizando la regla de la cadena, podemos calcular esta derivada:

$
\frac{dE}{dx_j} 
= \frac{dE}{dy} \frac{dy}{dx_j} \\
= \frac{dE}{dy} \frac{d (\sum_{i=1}^I w_i x_i)}{dx_j} \\
= \frac{dE}{dy} \sum_{i=1}^I \frac{d (w_i x_i) }{dx_j} \\
= \frac{dE}{dy} \frac{d (w_j x_j) }{dx_j} \\
= \frac{dE}{dy} w_j 
$

Entonces $\frac{dE}{dx_j} = \frac{dE}{dy} w_j$. Podemos generalizar esta definición, y calcular el gradiente respecto a todo el vector $x$ como:

$\frac{dE}{dx} = \frac{dE}{dy} w$


#### Notas

1. Es genial que la misma definición de $\frac{dE}{dx}$ funcione en ambos casos, ya sea con $1$ entrada o una cantidad $I$ arbitraria de ellas.
1. Es importante tener en cuenta que en este contexto podemos tratar a $\frac{dE}{dy}$ como una constante, ya que sus valores han sido previamente calculados.
1. Podríamos hacer la derivación de $\frac{dy}{dx}$ aparte, sin tomar en cuenta el error de la red, y luego obtener $\frac{dE}{dx}$ aplicando la regla de la cadena $\frac{dE}{dx} =\frac{dE}{dy} \frac{dy}{dx}$. No obstante, para ser más claros en el contexto del método `backward` una red, lo estamos haciendo todo al mismo tiempo.



### I entradas, O salidas

Nuevamente, vamos por la derivada de una de las entradas, es decir, $\frac{dE}{dx_j}$:

$
\frac{dE}{dx_j} = \frac{dE}{dy} \frac{dy}{dx_j} 
$

En este caso, $y$ es ahora un vector, con lo cual tenemos que sumar las contribuciones de cada elemento de $y$ a la regla de la cadena. Por ende:

$
\frac{dE}{dx_j} 
= \frac{dE}{dy} \frac{dy}{dx_j}  
= \sum_{i=1}^O \frac{dE}{dy_i} \frac{dy_i}{dx_j}
$

Ahora, sabemos que $y_i$ es el producto punto de la columna $i$ de $w$ con la entrada $x$, por la definición de la multiplicación de matrices. Entonces:

$
\frac{dE}{dx_j} 
= \sum_{i=1}^O \frac{dE}{dy_i} \frac{dy_i}{dx_j} \\
= \sum_{i=1}^O \frac{dE}{dy_i} \frac{d(w_{:,i} \cdot x)}{dx_j} \\
= \sum_{i=1}^O \frac{dE}{dy_i} \frac{d(\sum_{k=1}^I w_{k,i} x_k)}{dx_j} \\
= \sum_{i=1}^O \frac{dE}{dy_i} ( \sum_{k=1}^I \frac{d (w_{k,i} x_k)}{dx_j} ) \\
= \sum_{i=1}^O \frac{dE}{dy_i} w_{j,i}
$

Ahora, $\sum_{i=1}^O \frac{dE}{dy_i} w_{j,i}$ es simplemente el producto punto entre la columna $i$ de $w$ ($w_{:,i}$) y $\frac{dE}{dy}$. Entonces podemos escribir:

$
\frac{dE}{dx_j}  = \frac{dE}{dy} \cdot w_{:,i}
$

Generalizando para todo el vector $x$, si $\frac{dE}{dx_j}$ es el producto entre dos vectores, donde $j$ indica la columna de $w$, entonces podemos escribir $\frac{dE}{dx}$ como un producto entre el vector $\frac{dE}{dy}$ y la matriz $w$ entera:

$
\frac{dE}{dx} = w \frac{dE}{dy}
$

En este caso, el orden importa nuevamente. $w$ tiene tamaño $I \times O$ y $\frac{dE}{dy}$ tiene tamaño $O$, con lo cual $w \frac{dE}{dy}$ tiene tamaño $I$ (el mismo que $x$)




### Implementación por lotes

Para implementar la derivada para un lote de ejemplos podemos iterar sobre cada uno y calcular las derivadas como indicamos antes. Alternativamente, podemos reescribir la derivada para que funcione directamente para un lote de $N$ ejemplos (y por ende, de $N$ vectores de derivadas, tanto para la entrada como la salida)

En la implementación por lotes de $\frac{dE}{dx}$, tenemos que $x$ es una matriz de tamaño $N \times I$, y por ende también lo es $\frac{dE}{dx}$. Al mismo tiempo, como $\frac{dE}{dy}$ es en realidad $\frac{dE}{dx}$ de la capa siguiente, tenemos que $\frac{dE}{dy}$ es una matriz de tamaño $N \times O$.

Entonces, no podemos multiplicar $w \in R^{I \times O}$ por $\frac{dE}{dy} \in R^{N \times O}$. En este caso, puedes comprobar que la fórmula correcta es $\frac{dE}{dy} w^T$, ya que al multiplicar una matriz de tamaño  $N \times O$ por una de tamaño $O \times I$ ($w^T$), obtenemos una matriz de tamaño $N \times I$, o sea, del mismo tamaño de $x$:

$
\frac{dE}{dx} = \frac{dE}{dy} w^T
$

## $dE/dw$

En el caso del gradiente del error con respecto a $w$, también primero asumiremos que hay un solo ejemplo de entrada $x$, y vamos por casos de más simple a más complejo.



### 1 entrada, 1 salida

Este es el caso más simple, y es simétrico al de $\frac{dE}{dx}$:

$
\frac{dE}{dw} = \frac{dE}{dw} \frac{dw}{dx} = \frac{dE}{dw} \frac{d (wx)}{dw} = \frac{dE}{dy} x
$



### I entradas, 1 salida

Pasemos ahora al caso con $I$ entradas y 1 salida.

$y = x . w = \sum_{i=1}^I x_i w_i$

Como $w$ tiene $I$ elementos, entonces hay una derivada parcial por cada valor de $w$: $\frac{dE}{dw_j}$. Recordando que $\frac{dE}{dy}$ sigue siendo un escalar (porque hay una sola salida), y utilizando la regla de la cadena, podemos calcular esta derivada:

$
\frac{dE}{dw_j} 
= \frac{dE}{dy} \frac{dy}{dw_j} \\
= \frac{dE}{dy} \frac{d \sum_{i=1}^I w_i x_i }{dw_j} \\
= \frac{dE}{dy} \sum_{i=1}^I \frac{d (w_i x_i) }{dxw_j} \\
= \frac{dE}{dy} \frac{d (w_j x_j) }{dw_j} \\
= \frac{dE}{dy} x_j
$

Entonces $\frac{dE}{dw_j} = \frac{dE}{dy} x_j$. Podemos generalizar esta definición entonces, y calcular el gradiente respecto a todo el vector $x$ como:

$\frac{dE}{dw} = \frac{dE}{dy} x$

De nuevo, este caso es entonces **simétrico** con $x$, ya que $\frac{dE}{dx} = \frac{dE}{dy} w$.



### I entradas, O salidas

En este caso, al tener $O$ salidas, ahora vamos a tener que buscar la derivada de los pesos de cada entrada $i$ para cada salida $j$. En este caso, perdemos la simetría anterior (pero la recuperaremos en la versión por lotes).

Por ende, buscamos $\frac{dE}{dw_{i,j}}$. Por regla de la cadena:

$
\frac{dE}{dw_{i,j}}
= \frac{dE}{dy} \frac{dy}{dw_{i,j}}
= \frac{dE}{dy} \frac{d (xw)}{dw_{i,j}} 
$

Como $y$ es un vector, tenemos que sumar por todos sus valores para aplicar la regla de la cadena:

$
\frac{dE}{dw_{i,j}}
= \frac{dE}{dy} \frac{d (xw)}{dw_{i,j}}
= \sum_{k=1}^O \frac{dE}{dy_k} \frac{d(xw)_k}{dw_{i,j}} 
$

Como $y_k$ solo depende de $w_{i,j}$ si $j=k$, es decir, si estamos calculando la salida de la columna $k$, entonces:

$
\frac{dE}{dw_{i,j}}
= \frac{dE}{dy} \frac{d (xw)}{dw_{i,j}}
= \frac{dE}{dy_j} \frac{d(xw)_j}{dw_{i,j}} 
$

Por definición de la multiplicación de matrices, $(xw)_j = \sum_{l=1}^O x_l w_{l,j}$, o sea, multiplicamos $x$ por la columna $j$ de $w$. Reemplazando:

$
\frac{dE}{dw_{i,j}}
= \frac{dE}{dy_j} \frac{d(xw)_j}{dw_{i,j}} \\
= \frac{dE}{dy_j} \frac{d(\sum_{l=1}^O x_l w_{l,j})}{dw_{i,j}} \\
= \frac{dE}{dy_j} \sum_{l=1}^O \frac{d (x_l w_{l,j}) }{dw_{i,j}}
$

Como $w_{i,j}$ es solo un peso en particular de $w$, entonces de toda esa sumatoria solo queda el término que la contiene, es decir $\frac{d (x_i w_{i,j})}{w_{i,j}} = x_i$. Reemplazando:

$
\frac{dE}{dw_{i,j}} 
= \frac{dE}{dy_j} \sum_{l=1}^O \frac{d (x_l w_{l,j})}{dw_{i,j}} \\
= \frac{dE}{dy_j} \frac{d(x_i w_{i,j})}{d w_{i,j}} \\ 
= \frac{dE}{dy_j} x_i
$



### Expresión vectorial

La expresión anterior nos ayuda, pero deberíamos utilizar un loop `for` con índices `i` y `j` sobre toda la matriz de `w`. En lugar de eso, podemos generalizar entonces, observando el patrón de la matriz $\frac{dE}{dw}$:

$
\frac{dE}{dw} = \left(
\begin{matrix} 
    \frac{dE}{dy_1} x_1 & \frac{dE}{dy_2} x_1 & \dots & \frac{dE}{dy_O} x_1 \\
    \frac{dE}{dy_1} x_2 & \frac{dE}{dy_2} x_2 & \dots & \frac{dE}{dy_O} x_2 \\
    \vdots & \vdots & \ddots & \vdots \\
    \frac{dE}{dy_1} x_I & \frac{dE}{dy_2}  x_I & \dots & \frac{dE}{dy_O}  x_I \\
\end{matrix}
\right) = x \otimes \frac{dE}{dy}
$

Donde $\otimes$ es el [producto diádico o tensorial](https://es.wikipedia.org/wiki/Producto_tensorial) ([outer product](https://en.wikipedia.org/wiki/Outer_product) en inglés) entre dos vectores. En `numpy`, la función [`outer`](https://numpy.org/doc/stable/reference/generated/numpy.outer.html) permite hacer este tipo de operación sin loops.

Hay que tener en cuenta que el producto diádico *no* es conmutativo: si $a$ y $b$ tienen tamaño $p$ y $q$, entonces $ a \otimes b$ tiene tamaño $p \times q$, y $b \otimes a$ tiene tamaño $q \times p$. Por eso, como $\frac{dE}{dw}$ debe tener tamaño $I \times O$, entonces debemos computar $x \otimes\frac{dE}{dy}$ y no $\frac{dE}{dy} \otimes x$.



## Caso por lotes

En el caso de tener un lote de $N$ ejemplos, entonces recordamos que $x$ tiene tamaño $N \times I$, $w$ tiene tamaño $I \times O$, y $\frac{dE}{dy}$ tiene tamaño $N \times O$.

Al igual que en el caso de $b$, para calcular $\frac{dE}{dw}$ tenemos que sumar el gradiente que contribuye cada ejemplo $x_i$. Entonces:

$
\frac{dE}{dw} = \sum_{i=1}^{n} x_{i,:} \otimes \frac{dE}{dy_{i,:}}
$

Donde $x_{i,:}$ es la fila $i$ de $x$, es decir, el ejemplo $i$ (el equivalente en `numpy` sería `x[i,:]`)

Por ejemplo, si $N=2$, podemos verificar que:

$
\frac{dE}{dw} = x_{1,:}  \otimes \frac{dE}{dy_{1,:}} + x_{1,:}  \otimes \frac{dE}{dy_{2,:}}  \\
=  \left(
\begin{matrix}
    \frac{dE}{dy_{1,1}} x_{1,1} + \frac{dE}{dy_{2,1}} x_{2,1} & \frac{dE}{dy_{1,2}} x_{1,1} + \frac{dE}{dy_{2,2}} x_{2,1} & \dots & \frac{dE}{dy_{1,O}} x_{1,1} + \frac{dE}{dy_{2,O}} x_{2,1} \\
    \frac{dE}{dy_{1,1}} x_{1,2} + \frac{dE}{dy_{2,1}} x_{2,2} & \frac{dE}{dy_{1,2}} x_{1,2} + \frac{dE}{dy_{2,2}} x_{2,2} & \dots & \frac{dE}{dy_{1,O}} x_{1,2} + \frac{dE}{dy_{2,O}} x_{2,2} \\
    \vdots & \vdots & \ddots & \vdots \\
    \frac{dE}{dy_{1,1}} x_{1,I} + \frac{dE}{dy_{2,1}} x_{2,I} & \frac{dE}{dy_{1,2}} x_{1,I} + \frac{dE}{dy_{2,2}} x_{2,I} & \dots & \frac{dE}{dy_{1,O}} x_{1,I} + \frac{dE}{dy_{2,O}} x_{2,I} \\
\end{matrix}
\right) \\
 = x^t \frac{dE}{dy}
$

Esto también vale para cualquier $N$. Podemos confirmar esta identidad en base a los tamaños: si multiplicamos $x^t$ (tamaño $I \times N$) con $\frac{dE}{dy}$ (tamaño $N \times O$), obtenemos una matriz de tamaño $I \times O$, igual que $w$ y ¡justo el tamaño que debe tener $\frac{dE}{dw}$!.

Entonces, ahora si podemos ver la simetría entre las dos derivadas:

$
\frac{dE}{dw} = \frac{dy}{dw} \frac{dE}{dy} = x^t \frac{dE}{dy} \\
\frac{dE}{dx}= \frac{dy}{dx} \frac{dE}{dy} = \frac{dE}{dy} w
$
